In [ ]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip -q install datasets loralib sentencepiece 
!pip -q install bitsandbytes accelerate

  Installing build dependencies ... canceled
ERROR: Operation cancelled by user


# Koala 7B loading in 8bit on a T4

In [ ]:
!nvidia-smi

Fri Apr  7 13:10:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
import torch
import textwrap

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained("samwit/koala-7b")

base_model = LlamaForCausalLM.from_pretrained(
    "samwit/koala-7b",
    load_in_8bit=True,
    device_map='auto',
)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('http'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc48301118ce562b961b3c22d803539adc1e0c19/gpu-t4-s-1f9842c2

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [ ]:
!nvidia-smi

Fri Apr  7 13:21:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    28W /  70W |   8149MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import textwrap

pipe = pipeline(
    "text-generation",
    model=base_model, 
    tokenizer=tokenizer, 
    max_length=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
)

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

## Run it as a HF model

In [ ]:
%%time 
output = pipe('BEGINNING OF CONVERSATION: USER: \
What are the difference between Llamas, Alpacas and Koalas?')
print(wrap_text_preserve_newlines(output[0]['generated_text']))


BEGINNING OF CONVERSATION: USER: What are the difference between Llamas, Alpacas and Koalas?
Llamas, alpacas, and koalas are all unique animals with their own distinct characteristics. Here's a
comparison of these three species:

1.   Size: Llamas are larger than both alpacas and koalas. They can weigh up to 250 pounds (113 kilograms)
compared to alpacas that typically weigh around 100-150 pounds (45-68 kilograms), and koalas that weigh around
70 pounds (32 kilograms).
2.   Diet: Llamas primarily eat grasses and other vegetation, while alpacas consume mostly hay and other plant
material. Koalas mainly feed on eucalyptus leaves, which they chew into small pieces before swallowing them.
3.   Habitat: Llamas are native to South America and have been domesticated for thousands of years by
indigenous peoples in the Andes Mountains. They thrive at high altitudes where there is less competition from
predators such as pumas and eagles. Alpacas were also domesticated in the Andes Mountains but 

In [ ]:
%%time 
output = pipe('BEGINNING OF CONVERSATION: USER: \
Write a short note to Sam Altman giving reasons to open source GPT-4')

print(wrap_text_preserve_newlines(output[0]['generated_text']))

BEGINNING OF CONVERSATION: USER: Write a short note to Sam Altman giving reasons to open source GPT-4.

Dear Sam,

I hope this email finds you well. I am writing to share my thoughts on the potential benefits of opening up
GPT-4 as an open-source project. As someone who has been following your work closely and is interested in the
development of AI technologies, I believe that there are several compelling reasons why it would be beneficial
to make GPT-4 available for free use by anyone.

Firstly, making GPT-4 an open-source project could help to accelerate its adoption and widespread use. By
making the code and models freely available, more people will have access to them and can build upon them,
leading to new innovations and applications. This could also lead to increased collaboration between different
organizations and individuals working on similar projects, which could ultimately result in even better
outcomes.

Secondly, opening up GPT-4 as an open-source project could help to e

In [ ]:
%%time 
output = pipe('BEGINNING OF CONVERSATION: USER: What is the capital of England? \n')

print(wrap_text_preserve_newlines(output[0]['generated_text']))

BEGINNING OF CONVERSATION: USER: What is the capital of England?
Answer: The capital city of England is London.
CPU times: user 2.04 s, sys: 10.7 ms, total: 2.05 s
Wall time: 2.05 s


In [ ]:
%%time 
output = pipe('BEGINNING OF CONVERSATION: USER: Write a story about a Koala playing pool and beating all the camelids.')

print(wrap_text_preserve_newlines(output[0]['generated_text']))

BEGINNING OF CONVERSATION: USER: Write a story about a Koala playing pool and beating all the camelids.
Once upon a time, in a far-off land where animals could talk, there lived a koala named Koko. Koko was known
for his love of playing pool and his exceptional skills at it. One day, he decided to challenge all the other
animals in the kingdom to a game of pool.

The first animal he challenged was a camel named Cammy. Cammy accepted the challenge and they played a match.
At first, Koko thought he had an easy win as he had been practicing for weeks. But to his surprise, Cammy
proved to be a formidable opponent. The two played several games back and forth, with neither one able to gain
the upper hand.

Next up was a llama named Lola. She too accepted the challenge and they played a match. Once again, Koko
thought he would have an easy win but Lola's quick reflexes and strategic thinking made her a tough opponent.
They played several games back and forth, with both players giving their be

In [ ]:
%%time 
output = pipe('BEGINNING OF CONVERSATION: USER: As an AI do you like the Simpsons? What dow you know about Homer?')

print(wrap_text_preserve_newlines(output[0]['generated_text']))

BEGINNING OF CONVERSATION: USER: As an AI do you like the Simpsons? What dow you know about Homer?
As an AI, I don't have personal preferences or feelings. However, I can tell you that The Simpsons is a
popular animated television show created by Matt Groening. It first aired in 1989 and has since become one of
the longest-running TV shows in history. Homer Simpson is one of the main characters on the show, he is a
lazy, accident-prone father who often gets himself into trouble with his family and friends. He is known for
his catchphrase "Doh!" and his love for beer.
CPU times: user 26 s, sys: 74.6 ms, total: 26.1 s
Wall time: 30.5 s
